In [1]:
import os
import tensorflow as tf
import dask
import dask.array as da
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/segmentation/TGS Salt/dataset/test_y')
train_x, train_y, test_x, test_y

(dask.array<from-npy-stack, shape=(2400, 128, 128, 1), dtype=float32, chunksize=(2400, 128, 128, 1)>,
 dask.array<from-npy-stack, shape=(2400, 128, 128, 1), dtype=float32, chunksize=(2400, 128, 128, 1)>,
 dask.array<from-npy-stack, shape=(1600, 128, 128, 1), dtype=float32, chunksize=(1600, 128, 128, 1)>,
 dask.array<from-npy-stack, shape=(1600, 128, 128, 1), dtype=float32, chunksize=(1600, 128, 128, 1)>)

In [3]:
def build_block(input_layer, filters, norm=True, k=(3, 3)):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=k, padding='same', use_bias=not norm, kernel_initializer='glorot_normal')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_unet(n_filters=16, dropout=0.5):
    image_input = tf.keras.Input(shape=(128, 128, 1), name='input_layer')
    
    conv_1 = build_block(image_input, n_filters)
    conv_2 = build_block(conv_1, n_filters)
    pool_1 = tf.keras.layers.AveragePooling2D(padding='same')(conv_2)
    drop_1 = tf.keras.layers.SpatialDropout2D(dropout)(pool_1)
    
    conv_3 = build_block(drop_1, n_filters * 2)
    conv_4 = build_block(conv_3, n_filters * 2)
    pool_2 = tf.keras.layers.AveragePooling2D(padding='same')(conv_4)
    drop_2 = tf.keras.layers.SpatialDropout2D(dropout)(pool_2)
    
    conv_5 = build_block(drop_2, n_filters * 4)
    conv_6 = build_block(conv_5, n_filters * 4)
    pool_3 = tf.keras.layers.AveragePooling2D(padding='same')(conv_6)
    drop_3 = tf.keras.layers.SpatialDropout2D(dropout)(pool_3)
    
    conv_7 = build_block(drop_3, n_filters * 8)
    conv_8 = build_block(conv_7, n_filters * 8)
    pool_4 = tf.keras.layers.AveragePooling2D(padding='same')(conv_8)
    drop_4 = tf.keras.layers.SpatialDropout2D(dropout)(pool_4)
    
    conv_9 = build_block(drop_4, n_filters * 16)
    conv_10 = build_block(conv_9, n_filters * 16)
    
    upsp_1 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_10) #(-1, 16, 16, 256)
    upsp_1 = tf.keras.layers.concatenate([upsp_1, conv_8])#(-1, 16, 16, 384)
    upsp_1 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_1)#(-1, 16, 16, 384)
    conv_11 = build_block(upsp_1, n_filters * 8)#(-1, 16, 16, 128)
    conv_12 = build_block(conv_11, n_filters * 8)#(-1, 16, 16, 128)
    
    upsp_2 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_12) #(-1, 32, 32, 128)
    upsp_2 = tf.keras.layers.concatenate([upsp_2, conv_6]) #(-1, 32, 32, 192)
    upsp_2 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_2) #(-1, 32, 32, 192)
    conv_13 = build_block(upsp_2, n_filters * 4) #(-1, 32, 32, 64)
    conv_14 = build_block(conv_13, n_filters * 4) #(-1, 32, 32, 64)
    
    upsp_3 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_14) #(-1, 64, 64, 64)
    upsp_3 = tf.keras.layers.concatenate([upsp_3, conv_4]) #(-1, 64, 64, 96)
    upsp_3 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_3) #(-1, 64, 64, 96)
    conv_15 = build_block(upsp_3, n_filters * 2) #(-1, 64, 64, 32)
    conv_16 = build_block(conv_15, n_filters * 2) #(-1, 64, 64, 32)
    
    upsp_4 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_16) #(-1, 128, 128, 32)
    upsp_4 = tf.keras.layers.concatenate([upsp_4, conv_2])#(-1, 128, 128, 48)
    upsp_4 = tf.keras.layers.SpatialDropout2D(dropout)(upsp_4)#(-1, 128, 128, 48)
    conv_17 = build_block(upsp_4, n_filters)#(-1, 128, 128, 16)
    conv_18 = build_block(conv_17, n_filters)#(-1, 128, 128, 16)
    
    output = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer='glorot_normal', activation='sigmoid')(conv_18)
    model = tf.keras.Model(inputs=image_input, outputs=output)
    return model

In [4]:
model = build_unet()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 16) 144         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 16) 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [5]:
epochs, batch_size, lr, filepath = 10000, 16, 0.001, "day_1.weights.best.hdf5"
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))

data_gen_args = dict(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, vertical_flip=True)
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_x, augment=True, seed=seed)
mask_datagen.fit(train_y, augment=True, seed=seed)

image_generator = image_datagen.flow(x=train_x, batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(x=train_y, batch_size=batch_size, seed=seed)
train_generator = zip(image_generator, mask_generator)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow(x=test_x, y=test_y, batch_size=batch_size)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = tf.keras.callbacks.TensorBoard(log_dir=os.getcwd())

opt = tf.keras.optimizers.Adam() # 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=False,
                    workers=12,
                    shuffle=True,
                    #initial_epoch=65,
                    callbacks=[checkpoint, tb])

Epoch 1/10000
149/150 [============================>.] - ETA: 0s - loss: 9.6545 - acc: 0.6703
Epoch 00001: val_acc improved from -inf to 0.67616, saving model to day_1.weights.best.hdf5
150/150 [==============================] - 58s 388ms/step - loss: 9.6441 - acc: 0.6713 - val_loss: 9.9133 - val_acc: 0.6762
Epoch 2/10000
149/150 [============================>.] - ETA: 0s - loss: 7.9765 - acc: 0.7465
Epoch 00002: val_acc did not improve from 0.67616
150/150 [==============================] - 27s 183ms/step - loss: 7.9786 - acc: 0.7464 - val_loss: 15.7186 - val_acc: 0.2394
Epoch 3/10000
149/150 [============================>.] - ETA: 0s - loss: 7.3881 - acc: 0.7640
Epoch 00003: val_acc did not improve from 0.67616
150/150 [==============================] - 28s 186ms/step - loss: 7.3907 - acc: 0.7637 - val_loss: 11.8260 - val_acc: 0.3618
Epoch 4/10000
149/150 [============================>.] - ETA: 0s - loss: 7.0368 - acc: 0.7797
Epoch 00004: val_acc did not improve from 0.67616
150/150 

149/150 [============================>.] - ETA: 0s - loss: 5.8407 - acc: 0.8293
Epoch 00030: val_acc did not improve from 0.87846
150/150 [==============================] - 30s 198ms/step - loss: 5.8370 - acc: 0.8293 - val_loss: 4.7626 - val_acc: 0.8777
Epoch 31/10000
149/150 [============================>.] - ETA: 0s - loss: 5.7069 - acc: 0.8327
Epoch 00031: val_acc did not improve from 0.87846
150/150 [==============================] - 30s 198ms/step - loss: 5.6901 - acc: 0.8333 - val_loss: 5.1136 - val_acc: 0.8697
Epoch 32/10000
149/150 [============================>.] - ETA: 0s - loss: 5.7157 - acc: 0.8311
Epoch 00032: val_acc did not improve from 0.87846
150/150 [==============================] - 30s 198ms/step - loss: 5.7092 - acc: 0.8311 - val_loss: 4.7513 - val_acc: 0.8768
Epoch 33/10000
149/150 [============================>.] - ETA: 0s - loss: 5.7552 - acc: 0.8281
Epoch 00033: val_acc did not improve from 0.87846
150/150 [==============================] - 30s 198ms/step - los

149/150 [============================>.] - ETA: 0s - loss: 5.0545 - acc: 0.8489
Epoch 00059: val_acc did not improve from 0.89831
150/150 [==============================] - 30s 198ms/step - loss: 5.0488 - acc: 0.8491 - val_loss: 4.4435 - val_acc: 0.8821
Epoch 60/10000
149/150 [============================>.] - ETA: 0s - loss: 5.0799 - acc: 0.8489
Epoch 00060: val_acc improved from 0.89831 to 0.90066, saving model to day_1.weights.best.hdf5
150/150 [==============================] - 30s 199ms/step - loss: 5.0754 - acc: 0.8489 - val_loss: 3.9306 - val_acc: 0.9007
Epoch 61/10000
149/150 [============================>.] - ETA: 0s - loss: 4.9164 - acc: 0.8508
Epoch 00061: val_acc did not improve from 0.90066
150/150 [==============================] - 30s 198ms/step - loss: 4.9154 - acc: 0.8508 - val_loss: 4.1997 - val_acc: 0.8903
Epoch 62/10000
149/150 [============================>.] - ETA: 0s - loss: 5.1427 - acc: 0.8439
Epoch 00062: val_acc improved from 0.90066 to 0.90215, saving model 

149/150 [============================>.] - ETA: 0s - loss: 4.4990 - acc: 0.8645
Epoch 00088: val_acc did not improve from 0.90875
150/150 [==============================] - 30s 198ms/step - loss: 4.5050 - acc: 0.8642 - val_loss: 4.1022 - val_acc: 0.8938
Epoch 89/10000
149/150 [============================>.] - ETA: 0s - loss: 4.5813 - acc: 0.8621
Epoch 00089: val_acc did not improve from 0.90875
150/150 [==============================] - 30s 198ms/step - loss: 4.5824 - acc: 0.8619 - val_loss: 3.8300 - val_acc: 0.8971
Epoch 90/10000
149/150 [============================>.] - ETA: 0s - loss: 4.5354 - acc: 0.8635
Epoch 00090: val_acc did not improve from 0.90875
150/150 [==============================] - 30s 198ms/step - loss: 4.5367 - acc: 0.8633 - val_loss: 3.9981 - val_acc: 0.8918
Epoch 91/10000
149/150 [============================>.] - ETA: 0s - loss: 4.5852 - acc: 0.8611
Epoch 00091: val_acc did not improve from 0.90875
150/150 [==============================] - 30s 198ms/step - los

Epoch 118/10000
149/150 [============================>.] - ETA: 0s - loss: 4.1693 - acc: 0.8726
Epoch 00118: val_acc did not improve from 0.91621
150/150 [==============================] - 30s 198ms/step - loss: 4.1633 - acc: 0.8728 - val_loss: 3.5243 - val_acc: 0.9023
Epoch 119/10000
149/150 [============================>.] - ETA: 0s - loss: 4.3053 - acc: 0.8698
Epoch 00119: val_acc did not improve from 0.91621
150/150 [==============================] - 30s 198ms/step - loss: 4.2939 - acc: 0.8699 - val_loss: 3.1602 - val_acc: 0.9159
Epoch 120/10000
149/150 [============================>.] - ETA: 0s - loss: 4.1491 - acc: 0.8733
Epoch 00120: val_acc did not improve from 0.91621
150/150 [==============================] - 30s 198ms/step - loss: 4.1592 - acc: 0.8730 - val_loss: 3.2386 - val_acc: 0.9112
Epoch 121/10000
149/150 [============================>.] - ETA: 0s - loss: 4.3721 - acc: 0.8669
Epoch 00121: val_acc did not improve from 0.91621
150/150 [==============================] - 3

...

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_generator.py", line 171, in fit_generator
    x, y, sample_weight=sample_weight, class_weight=class_weight)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1828, in train_on_batch
    outputs = self.train_function(ins)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/backend.py", line 2978, in __call__
    run_metadata=self.run_metadata)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1399, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/s

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_generator.py", line 171, in fit_generator
    x, y, sample_weight=sample_weight, class_weight=class_weight)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1828, in train_on_batch
    outputs = self.train_function(ins)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/backend.py", line 2978, in __call__
    run_metadata=self.run_metadata)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1399, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/s

TypeError: must be str, not list